In [2]:
import pickle
import random
import time
import math

class Keller5TurboMegaShock:
    def __init__(self, watchdog_seconds=200):
        self.DIMENSION = 5
        self.TARGET_SIZE = 28
        self.WATCHDOG_LIMIT = watchdog_seconds
        
        # Hyperparameters
        self.STAGNANT_LIMIT = 600   # Impatient trigger
        self.SHOCK_THRESHOLD = 3    # Mega-shock every 3rd reheat
        self.SAMPLE_SIZE = 600
        self.TEMP_PUSH = 45.0
        self.COOLING_PUSH = 0.9999
        
        # Tracking variables
        self.reheat_count = 0
        
        # Load Data
        filename = f'keller_codes_d{self.DIMENSION}.data'
        try:
            with open(filename, 'rb') as f:
                self.codes_int = pickle.load(f)
        except FileNotFoundError:
            print(f"❌ Error: {filename} not found in the current directory.")
            return

        self.n = len(self.codes_int)
        
        # Pre-calculated Exp Table for performance
        self.exp_table = [math.exp(i) for i in range(31)]

    def are_neighbors(self, u: int, v: int) -> bool:
        a, b = (u, v) if u < v else (v, u)
        return ((self.codes_int[a] >> b) & 1) == 0

    def get_vertex_conflicts(self, v, current_clique):
        return sum(1 for u in current_clique if u != v and not self.are_neighbors(v, u))

    def compute_total_conflicts(self, clique):
        count = 0
        for i in range(len(clique)):
            for j in range(i + 1, len(clique)):
                if not self.are_neighbors(clique[i], clique[j]):
                    count += 1
        return count

    def solve(self):
        start_time = time.time()
        seed = int(time.time() * 1000) % 2**32
        random.seed(seed)
        
        print(f"🚀 Turbo Mega-Shock Active | Seed: {seed} | Limit: {self.WATCHDOG_LIMIT}s")
        print("-" * 60)
        
        # Init: Start at target size immediately
        current_clique = random.sample(range(self.n), self.TARGET_SIZE)
        current_conflict = self.compute_total_conflicts(current_clique)
        
        best_clique = current_clique.copy()
        best_conflict = current_conflict
        
        temp = self.TEMP_PUSH
        stagnant = 0
        iteration = 0
        last_heartbeat = time.time()

        while best_conflict > 0:
            iteration += 1
            elapsed = time.time() - start_time
            
            if elapsed > self.WATCHDOG_LIMIT:
                print(f"\n🛑 Watchdog Triggered ({elapsed:.1f}s). Final Best Conflicts: {best_conflict}")
                return None, seed, elapsed

            # Heartbeat Monitor (Standard Jupyter output)
            if time.time() - last_heartbeat > 3.0:
                print(f"  [Progress] Time: {elapsed:5.1f}s | Best Conf: {best_conflict} | Temp: {temp:.2f} | Reheats: {self.reheat_count}")
                last_heartbeat = time.time()

            # 1. Stochastic Selection (Soft-max weighted removal)
            v_confs = [self.get_vertex_conflicts(v, current_clique) for v in current_clique]
            weights = [self.exp_table[min(c, 30)] for c in v_confs]
            worst_idx = random.choices(range(len(current_clique)), weights=weights, k=1)[0]
            
            removed_v_conflict = v_confs[worst_idx]
            temp_set = current_clique[:worst_idx] + current_clique[worst_idx+1:]
            
            # 2. Candidate Sampling (Best-fit approach)
            candidates = random.sample(range(self.n), self.SAMPLE_SIZE)
            best_rep, min_c = -1, 999
            for v in candidates:
                if v in temp_set: continue
                c = self.get_vertex_conflicts(v, temp_set)
                if c < min_c:
                    min_c, best_rep = c, v
                if c == 0: break
            
            # 3. Delta Logic
            new_conflict = current_conflict - removed_v_conflict + min_c
            delta = new_conflict - current_conflict
            
            # 4. Metropolis Acceptance
            if delta <= 0 or (temp > 1e-8 and random.random() < math.exp(-delta / temp)):
                current_clique = temp_set + [best_rep]
                current_conflict = new_conflict
                
                if current_conflict < best_conflict:
                    best_clique, best_conflict = current_clique.copy(), current_conflict
                    stagnant = 0
                    self.reheat_count = 0 
                else:
                    stagnant += 1
            else:
                stagnant += 1

            # 5. MEGA-SHOCK Logic
            if stagnant > self.STAGNANT_LIMIT:
                self.reheat_count += 1
                if self.reheat_count >= self.SHOCK_THRESHOLD:
                    print(f"  ⚡ MEGA-SHOCK! Displacing 4 vertices at {elapsed:.1f}s...")
                    for _ in range(4):
                        idx = random.randint(0, self.TARGET_SIZE - 1)
                        current_clique[idx] = random.randint(0, self.n - 1)
                    current_conflict = self.compute_total_conflicts(current_clique)
                    temp = self.TEMP_PUSH 
                    self.reheat_count = 0
                else:
                    temp = self.TEMP_PUSH * 0.7 
                stagnant = 0

            temp *= self.COOLING_PUSH

        # Verification & Notebook Display
        duration = time.time() - start_time
        self.display_notebook_results(best_clique, seed, duration)
        return best_clique, seed, duration

    def display_notebook_results(self, clique, seed, duration):
        print("\n" + "="*60)
        print("✅ SUCCESS: MAX CLIQUE FOUND")
        print("="*60)
        print(f"Runtime:      {duration:.2f} seconds")
        print(f"Seed used:    {seed}")
        print(f"Clique Size:  {len(clique)}")
        print("-" * 60)
        print("Vertices in Clique:")
        print(sorted(clique))
        print("="*60)

# --- Execution for Jupyter ---
solver = Keller5TurboMegaShock(watchdog_seconds=300)
clique, seed, runtime = solver.solve()

🚀 Turbo Mega-Shock Active | Seed: 1245586157 | Limit: 300s
------------------------------------------------------------
  [Progress] Time:   3.0s | Best Conf: 5 | Temp: 43.93 | Reheats: 0
  [Progress] Time:   6.0s | Best Conf: 5 | Temp: 42.60 | Reheats: 0
  [Progress] Time:   9.1s | Best Conf: 4 | Temp: 41.25 | Reheats: 0
  [Progress] Time:  12.1s | Best Conf: 3 | Temp: 40.00 | Reheats: 0
  [Progress] Time:  15.1s | Best Conf: 2 | Temp: 38.64 | Reheats: 0
  [Progress] Time:  18.1s | Best Conf: 2 | Temp: 31.22 | Reheats: 1
  [Progress] Time:  21.1s | Best Conf: 2 | Temp: 30.13 | Reheats: 1
  [Progress] Time:  24.1s | Best Conf: 2 | Temp: 30.91 | Reheats: 2
  [Progress] Time:  27.1s | Best Conf: 2 | Temp: 29.77 | Reheats: 2
  ⚡ MEGA-SHOCK! Displacing 4 vertices at 27.4s...
  [Progress] Time:  30.1s | Best Conf: 2 | Temp: 43.57 | Reheats: 0
  [Progress] Time:  33.1s | Best Conf: 2 | Temp: 31.27 | Reheats: 1
  [Progress] Time:  36.1s | Best Conf: 2 | Temp: 30.20 | Reheats: 1
  [Progress] T